In [1]:
import numpy as np
# import trimesh as tr
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
"""Be sure to write down the 'movement' vector that gets print out!"""
name = '' # name of point cloud
pc_raw = o3d.io.read_point_cloud(f"{name}.ply")
pc_raw = np.asarray(pc_raw.points)
print(pc_raw.shape)

def subtract_mean(array):
    x = np.mean(array[:, 0])
    y = np.mean(array[:, 1])
    z = np.mean(array[:, 2])
    array[:, 0] = array[:, 0] - x
    array[:, 1] = array[:, 1] - y
    array[:, 2] = array[:, 2] - z
    displacement = np.asarray([x, y, z])
    return array, displacement

moved, movement = subtract_mean(pc_raw)
print(f"movement: {movement}")
print(f"largest number: {np.max(moved)}")
print(f"smallest number: {np.min(moved)}")
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(moved)
o3d.io.write_point_cloud(f"{name}_moved.ply", pc)

In [ ]:
o3d.geometry.KDTreeSearchParam.KNNSearch = 10
pc.estimate_normals()
pc.orient_normals_to_align_with_direction(
    orientation_reference=np.asarray([0.0, 0.0, 1.0]))
o3d.io.write_point_cloud(f"{name}_normals_fixed.ply", pc)

In [2]:
"""Optional: load existing point cloud"""
# pc = o3d.io.read_point_cloud("tgb_231205_normals_fixed.ply")

In [4]:
depth = 11
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
     pc, depth=depth)
remove_vertices_mask = densities < np.quantile(densities, 0.005)
mesh.remove_vertices_by_mask(remove_vertices_mask)
o3d.io.write_triangle_mesh(f"{name}_mesh_d={depth}.ply", mesh)

True